In [3]:
#svm
import menpo.io as mio
import os
from sklearn import svm
import numpy as np
from sklearn.decomposition import PCA

path_to_svm_training_database = '/Programing/GR/Code/CK+/aam-images/**/**/**/*'

class ChangeVector:
    def __init__(self, facs = [], landmarkChange = []):
        self.landmarkChange = landmarkChange
        self.facs = facs

def process(image, crop_proportion=0.2, max_diagonal=400):
    if image.n_channels == 3:
        image = image.as_greyscale()
    image = image.crop_to_landmarks_proportion(crop_proportion)
    d = image.diagonal()
    if d > max_diagonal:
        image = image.rescale(float(max_diagonal) / d)
    return image

#process changingVector, reduce dimension from 2x68 to 1x68, by process PCA
def pca(changeVector):
    X = np.array(changeVector)
    pca_model = PCA(n_components=1)
    return pca_model.fit_transform(X)


training_images = mio.import_images(path_to_svm_training_database, verbose=True)
training_images = training_images.map(process)

path_to_facs = '/Programing/GR/Code/CK+/FACS/'
path_to_emotions = '/Programing/GR/Code/CK+/Emotion/'

#create training data
count = 0;
svm_training_data = []
while(count < len(training_images)):
    file_path = str(training_images[count].path).split("\\")
    
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = []
    for line in fi: # read rest of lines
        for x in line.split():
            if(int(float(x)) not in data_facs and int(float(x))!= 0):
                data_facs.append(int(float(x)))
    #print(array)
    fi.close()
    
    landmarkChange = []
    landmark_neutral = training_images[count].landmarks['PTS'].lms.points
    landmark_perk = training_images[count + 1].landmarks['PTS'].lms.points
    for i in range(0,68):
        landmarkChange.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])
    
    svm_training_data.append(ChangeVector(data_facs, landmarkChange))   
    count = count + 2


    

facs = []

#create facs array
for data in svm_training_data:
    for facs_code in data.facs:
        if(int(facs_code) not in facs and int(facs_code)!= 0):
            facs.append(facs_code)

facs.sort()



Found 1146 assets, index the returned LazyList to import.


In [112]:
#create model for each action unit in facs[]
from sklearn.preprocessing import normalize
models = []
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_training_data:
        if(au in data.facs):
            y_label.append(1)
        else:
            y_label.append(0)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmarkChange:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
    clf = svm.LinearSVR()
    clf.fit(normalize(x_training), y_label)
    models.append(clf)


In [109]:
print(len(x_training[0]))
#svm_training_data[2].landmarkChange

136


In [25]:
#create testing data
svm_testing_data = []
path_to_svm_testing_database = "/Programing/GR/Code/CK+/test-aam-images/**/**/**/*"
testing_images = mio.import_images(path_to_svm_testing_database, verbose=True)
testing_images = testing_images.map(process)

count = 0;
while(count < len(testing_images)):
    file_path = str(testing_images[count].path).split("\\")
    
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = []
    for line in fi: # read rest of lines
        for x in line.split():
            if(int(float(x)) not in data_facs and int(float(x)) != 0):
                data_facs.append(int(float(x)))
    #print(array)
    fi.close()
    
    landmarkChange = []
    landmark_neutral = testing_images[count].landmarks['PTS'].lms.points
    landmark_perk = testing_images[count + 1].landmarks['PTS'].lms.points
    for i in range(0,68):
        landmarkChange.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])
    
    svm_testing_data.append(ChangeVector(data_facs, landmarkChange))   
    count = count + 2



Found 42 assets, index the returned LazyList to import.


In [28]:
print(facs)

[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 38, 39, 43, 44, 45, 54, 61, 62, 63, 64]


In [81]:
data = svm_training_data[0]
vector = []
for tmp in data.landmarkChange:
    vector.append(tmp[0])
    vector.append(tmp[1])
for model in models:
    if(model.predict([vector]) == 0):
        print(facs[models.index(model)])
#print(models[1].predict([vector]))
print(data.facs)

1
2
3
4
5
6
7
9
10
11
12
13
14
15
16
17
18
20
21
22
23
24
25
26
27
28
29
30
31
34
38
39
43
44
45
54
61
62
63
64
[9, 4, 17, 2]


In [113]:
#regresssion
wrong_predict = 0
for data in svm_testing_data:
    print('#####')
    local_wrong_predict = 0
    tmp = []
    predict = []
    for vector in data.landmarkChange:
        tmp.append(vector[0])
        tmp.append(vector[1])
        
    for model in models:
        if(model.predict([tmp]) >= 0.5):
            predict.append([facs[models.index(model)], model.predict([tmp])[0]])
            #print(facs[models.index(model)])
            if(facs[models.index(model)] not in data.facs):
                local_wrong_predict += 1
        if(model.predict([tmp]) == 0):
            if(facs[models.index(model)] in data.facs):
                local_wrong_predict += 1
    print(predict)
    print("---")
    for gt_facs in data.facs:
        print([gt_facs, models[facs.index(gt_facs)].predict([tmp])[0]])
    wrong_predict += local_wrong_predict
    
print(wrong_predict)

#####
[[4, 1.0241648594920529], [6, 1.1129131389552569], [7, 0.87741194238429165], [9, 1.3736887337537091], [12, 4.63203871989219], [15, 23.488560120787977], [17, 15.582304719276816], [25, 11.997206798738116]]
---
#####
[[6, 3.2660237421587093], [12, 22.218326973494786]]
---
[12, 22.218326973494786]
[2, -6.6977203680381576]
#####
[[25, 27.74129375772625]]
---
[25, 27.74129375772625]
[26, 0.0091655883355109182]
#####
[[4, 11.429836951058279], [6, 3.2939181634175001], [12, 14.658760932683299], [15, 1.1768223462469016], [17, 16.259837550931024]]
---
[4, 11.429836951058279]
[7, 0.21318092462056112]
[14, 0.0018758150727534184]
[17, 16.259837550931024]
[24, -0.0011604070263122736]
[39, 5.1509133532805592e-05]
#####
[[6, 28.966962643101422], [12, 60.416220179845205], [25, 64.006754835832567]]
---
[4, -7.40431341768649]
[6, 28.966962643101422]
[7, 0.043420118940302586]
[9, 0.036250990625251653]
[10, 0.0011895957635564901]
[12, 60.416220179845205]
[20, 0.010905029543725206]
[25, 64.006754835832